# Experiment x-2
Energy-based Flow Classifier (EFC) - Pure Python
Applying EFC to Bitcoin blockchain in the presence of label scarcity

In [7]:
import os
import sys

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

In [4]:
from efc_python.classification_functions import (
    one_class_fit,
    one_class_predict,
)
from efc_python.generic_discretize import discretize, get_intervals

In [5]:
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    roc_auc_score,
    confusion_matrix
)

## Dataset

In [8]:
ROOT_DIR = os.getcwd()
sys.path.insert(0, ROOT_DIR)

In [9]:
def train_test_split(X, y, train_test_idx):
    X_train_df = X.loc[train_test_idx["train"]]
    X_test_df = X.loc[train_test_idx["test"]]

    y_train = y.loc[train_test_idx["train"]]
    y_test = y.loc[train_test_idx["test"]]

    return X_train_df, X_test_df, y_train, y_test

In [10]:
def setup_train_test_idx(
    X, last_train_time_step, last_time_step, aggregated_timestamp_column="time_step"
):
    """The aggregated_time_step_column needs to be a column with integer values, such as year, month or day"""

    split_timesteps = {}

    split_timesteps["train"] = list(range(last_train_time_step + 1))
    split_timesteps["test"] = list(range(last_train_time_step + 1, last_time_step + 1))

    train_test_idx = {}
    train_test_idx["train"] = X[
        X[aggregated_timestamp_column].isin(split_timesteps["train"])
    ].index
    train_test_idx["test"] = X[
        X[aggregated_timestamp_column].isin(split_timesteps["test"])
    ].index

    return train_test_idx

In [11]:
def combine_dataframes(df_classes, df_features, only_labeled=True):
    df_combined = pd.merge(
        df_features, df_classes, left_on="id", right_on="txId", how="left"
    )
    if only_labeled == True:
        df_combined = df_combined[df_combined["class"] != 2].reset_index(drop=True)
    df_combined.drop(columns=["txId"], inplace=True)
    return df_combined

In [12]:
def rename_classes(df_classes):
    df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)
    return df_classes

In [13]:
def rename_features(df_features):
    df_features.columns = (
        ["id", "time_step"]
        + [f"trans_feat_{i}" for i in range(93)]
        + [f"agg_feat_{i}" for i in range(72)]
    )
    return df_features

In [14]:
def import_elliptic_data_from_csvs():
    df_classes = pd.read_csv(
        os.path.join(ROOT_DIR, "datasets/elliptic/elliptic_txs_classes.csv")
    )
    df_edges = pd.read_csv(
        os.path.join(ROOT_DIR, "datasets/elliptic/elliptic_txs_edgelist.csv")
    )
    df_features = pd.read_csv(
        os.path.join(ROOT_DIR, "datasets/elliptic/elliptic_txs_features.csv"),
        header=None,
    )
    return df_classes, df_edges, df_features

In [15]:
def load_elliptic_data(only_labeled=True, drop_node_id=True):
    df_classes, df_edges, df_features = import_elliptic_data_from_csvs()
    df_features = rename_features(df_features)
    df_classes = rename_classes(df_classes)
    df_combined = combine_dataframes(df_classes, df_features, only_labeled)

    if drop_node_id == True:
        X = df_combined.drop(columns=["id", "class"])
    else:
        X = df_combined.drop(columns="class")

    y = df_combined["class"]

    return X, y

In [16]:
def run_elliptic_preprocessing_pipeline(
    last_train_time_step, last_time_step, only_labeled=True, drop_node_id=True
):
    X, y = load_elliptic_data(only_labeled, drop_node_id)
    train_test_idx = setup_train_test_idx(X, last_train_time_step, last_time_step)
    X_train_df, X_test_df, y_train, y_test = train_test_split(X, y, train_test_idx)

    return X_train_df, X_test_df, y_train, y_test

## EFC

In [19]:
# Elliptic data set timestep
last_time_step = 49
last_train_time_step = 34
only_labeled = True

In [30]:
# '1': 1, -> class1 (illicit)
# '2': 0, -> class2 (licit)
# 'unknown': 2 -> dropped
X_train_df, X_test_df, y_train, y_test = run_elliptic_preprocessing_pipeline(
    last_train_time_step=last_train_time_step,
    last_time_step=last_time_step,
    only_labeled=only_labeled,
)

run_elliptic_preprocessing_pipeline True


/var/folders/8b/c9dvlw851kx2vf3r7gc5gbbh0000gn/T/ipykernel_35367/3533226547.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)


In [31]:
results_efc = 'results/efc/experiment-x-2'

In [32]:
# make experiments results dir if not exists
if not os.path.exists(results_efc):
    os.makedirs(results_efc)

In [33]:
sizes = {
    "X Size": len(X_train_df) + len(X_test_df),
    "y Size": len(y_train) + len(y_test),
    "X_train Size": len(X_train_df),
    "X_test Size": len(X_test_df),
    "y_train Size": len(y_train),
    "y_test Size": len(y_test),
    "y_train Malicious Size": len(np.where(y_train == 1)[0]),
    "y_train Bening Size": len(np.where(y_train == 0)[0]),
    "y_test Malicious Size": len(np.where(y_test == 1)[0]),
    "y_test Bening Size": len(np.where(y_test == 0)[0]),
}

In [34]:
df_sizes = pd.DataFrame(data=sizes, index=[0])

In [35]:
df_sizes

,X Size,y Size,X_train Size,X_test Size,y_train Size,y_test Size,y_train Malicious Size,y_train Bening Size,y_test Malicious Size,y_test Bening Size
0,46564,46564,29894,16670,29894,16670,3462,26432,1083,15587


In [36]:
intervals = get_intervals(X_train_df, 10)  # get discretization intervals from train set

In [37]:
X_train_df = discretize(X_train_df, intervals)  # discretize train
X_test_df = discretize(X_test_df, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29894   NaN
29895   NaN
29896   NaN
29897   NaN
29898   NaN
    

In [38]:
idx_abnormal = np.where(y_train == 1)[0]  # find abnormal samples indexes in the training set

In [39]:
X_train_df.drop(idx_abnormal, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)

In [40]:
y_train.drop(idx_abnormal, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [44]:
# EFC's hyperparameters
Q = int(X_test.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [45]:
Q

287

In [46]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train_df), Q, LAMBDA)  # train model

  0%|                                                                                                        | 0/166 [00:00<?, ?it/s]/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/efc_python/dca_functions.py:74: RuntimeWarning: divide by zero encountered in scalar divide
  fields[i * (Q - 1) + ai] /= (
100%|██████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [15:14<00:00,  5.51s/it]
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/efc_python/classification_functions.py:38: RuntimeWarning: divide by zero encountered in log
  c_matrix = np.log(c_matrix)
  0%|                                                                                                      | 0/26432 [00:00<?, ?it/s]/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/efc_python/classification_functions.py:24: RuntimeWarning: invalid value encountered in scalar subtract
  e -=

In [ ]:
y_predicted, y_energies = one_class_predict(np.array(X_test_df), coupling, h_i, cutoff, Q)  # test model

  0%|                                                                                                        | 0/165 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:00<00:00, 651313.81it/s]
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc/efc_python/classification_functions.py:61: RuntimeWarning: invalid value encountered in scalar subtract
  e -= h_i[j * (Q - 1) + j_value]


100%|█████████████████████████████████████████████████████████████████████████████████████████| 165/165 [00:00<00:00, 1116226.06it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████| 164/164 [00:00<00:00, 685808.43it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████| 163/163 [00:00<00:00, 1109856.42it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████| 162/16

In [48]:
# ploting energies
benign = np.where(y_test == 0)[0]
malicious = np.where(y_test == 1)[0]

benign_energies = y_energies[benign]
malicious_energies = y_energies[malicious]
cutoff = clf.estimators_[0].cutoff_

bins = np.histogram(y_energies, bins=60)[1]

plt.hist(
    malicious_energies,
    bins,
    facecolor="#006680",
    alpha=0.7,
    ec="white",
    linewidth=0.3,
    label="malicious",
)
plt.hist(
    benign_energies,
    bins,
    facecolor="#b3b3b3",
    alpha=0.7,
    ec="white",
    linewidth=0.3,
    label="benign",
)
plt.axvline(cutoff, color="r", linestyle="dashed", linewidth=1)
plt.legend()

plt.xlabel("Energy", fontsize=12)
plt.ylabel("Density", fontsize=12)

plt.show()
plt.savefig(f'{results_efc}/1_labeled_samples.png')

NameError: name 'y_energies' is not defined

In [ ]:
def calculate_model_score(y_true, y_pred):
    metric_dict = {
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, average="weighted"),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred, average="weighted"),
        "recall": recall_score(y_true, y_pred, average="weighted"),
    }
    return metric_dict

In [131]:
model_score = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")

In [132]:
df_efc_metrics = pd.DataFrame(data=model_score, index=[0])

np.float64(0.06829857963130856)

In [ ]:
df_efc_metrics

In [ ]:
labels_cm = ["True Negative", "False positive", "False Negative", "True Positive"]

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=[1, 0])

In [ ]:
cm = np.reshape(cm, -1).tolist()

In [ ]:
cm

In [ ]:
confusion_matrix = {'Technique': 'All samples'} | {label: val for val, label in zip(cm, labels_cm)}

In [ ]:
confusion_matrix

### Train EFC With 5% Labeled Elliptic Data Set

In [184]:
# elliptic data set from reaml repo
X_train_5, X_test_5, y_train_5, y_test_5 = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/research_aml_elliptic/src/experiments/general_functions/elliptic_data_preprocessing.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)


In [185]:
# efc preps
intervals = get_intervals(X_train_5, 10)  # get discretization intervals from train set
X_train_5 = discretize(X_train_5, intervals)  # discretize train
X_test_5 = discretize(X_test_5, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29894   NaN
29895   NaN
29896   NaN
29897   NaN
29898   NaN
         ..


In [186]:
indices_illicit = np.where(y_train_5 == 1)[0]

In [187]:
drop_indices_illicit = np.random.choice(indices_illicit, size=ceil(len(indices_illicit) * 0.95))

In [188]:
# retrieve idxs abnormals and choose 95% of them
# abnormals == class1 (illicit)
# drop random labeled indices
X_train_5.drop(drop_indices_illicit, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)
y_train_5.drop(drop_indices_illicit, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [189]:
# EFC's hyperparameters
Q = np.int64(X_test.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [190]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train_5), Q, LAMBDA)  # train model

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:01<00:00, 87.88it/s]


In [192]:
y_predicted_5, energies = one_class_predict(np.array(X_test_5), coupling, h_i, cutoff, Q)  # test model

In [201]:
# colect results
confusion_matrix_5 = confusion_matrix(y_test_5, y_predicted_5)
print("Single-class results")
print('confusion_matrix', confusion_matrix_5)

Single-class results
confusion_matrix [[10273  5314]
 [  891   192]]


In [202]:
model_score_5 = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")

In [203]:
y_true_5 = y_test_5.values
y_pred_5 = y_predicted_5

In [204]:
metric_dict_5 = {
        "contamination": "5%",
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
        "model_score": model_score_5,
    }

In [205]:
pprint(metric_dict_5)

{'accuracy': 0.6301139772045591,
 'contamination': '5%',
 'f1': np.float64(0.06829857963130856),
 'f1_macro': np.float64(0.4187761889998471),
 'f1_micro': np.float64(0.6301139772045591),
 'model_score': np.float64(0.06829857963130856),
 'precision': np.float64(0.04083107497741644),
 'recall': np.float64(0.20867959372114497),
 'roc_auc': np.float64(0.4340376219712416)}


In [208]:
df_efc_metrics.loc[1] = metric_dict_5

In [209]:
df_efc_metrics

,contamination,accuracy,f1,f1_micro,f1_macro,precision,recall,roc_auc,model_score
0,100%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299
1,5%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299


In [214]:
confusion_matrix_5 = {
            "contamination": "5%",
            "True Negative": 10273,
            "False positive": 5314,
            "False Negative": 891,
            "True Positive": 192
}

In [215]:
df_efc_confusion_matrix.loc[1] = confusion_matrix_5

In [216]:
df_efc_confusion_matrix

,contamination,True Negative,False positive,False Negative,True Positive
0,100%,10278,5309,857,226
1,5%,10273,5314,891,192


### Train EFC With 10% Labeled Elliptic Data Set

In [229]:
# elliptic data set from reaml repo
X_train_10, X_test_10, y_train_10, y_test_10 = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/research_aml_elliptic/src/experiments/general_functions/elliptic_data_preprocessing.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)


In [230]:
# efc preps
intervals = get_intervals(X_train_10, 10)  # get discretization intervals from train set
X_train_10 = discretize(X_train_10, intervals)  # discretize train
X_test_10 = discretize(X_test_10, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29894   NaN
29895   NaN
29896   NaN
29897   NaN
29898   NaN
         ..


In [235]:
indices_illicit_10 = np.where(y_train_10 == 1)[0]

In [236]:
drop_indices_illicit_10 = np.random.choice(indices_illicit_10, size=ceil(len(indices_illicit_10) * 0.90))

In [237]:
# retrieve idxs abnormals and choose 90% of them
# abnormals == class1 (illicit)
# drop random labeled indices
X_train_10.drop(drop_indices_illicit_10, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)
y_train_10.drop(drop_indices_illicit_10, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [238]:
# EFC's hyperparameters
Q = np.int64(X_test_10.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [239]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train_10), Q, LAMBDA)  # train model

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:01<00:00, 88.29it/s]


In [240]:
y_predicted_10, energies = one_class_predict(np.array(X_test_10), coupling, h_i, cutoff, Q)  # test model

In [241]:
# colect results
confusion_matrix_10 = confusion_matrix(y_test_10, y_predicted_10)
print("Single-class results")
print('confusion_matrix', confusion_matrix_10)

Single-class results
confusion_matrix [[10291  5296]
 [  891   192]]


In [242]:
model_score_10 = calculate_model_score(y_true=y_test_10.values, y_pred=y_predicted_10, metric="f1")

In [243]:
y_true_10 = y_test_10.values
y_pred_10 = y_predicted_10

In [244]:
metric_dict_10 = {
        "contamination": "10%",
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
        "model_score": model_score_10,
    }

In [245]:
pprint(metric_dict)

{'accuracy': 0.6301139772045591,
 'contamination': '100%',
 'f1': np.float64(0.06829857963130856),
 'f1_macro': np.float64(0.4187761889998471),
 'f1_micro': np.float64(0.6301139772045591),
 'model_score': np.float64(0.06829857963130856),
 'precision': np.float64(0.04083107497741644),
 'recall': np.float64(0.20867959372114497),
 'roc_auc': np.float64(0.4340376219712416)}


In [246]:
df_efc_metrics.loc[2] = metric_dict_10

In [247]:
df_efc_metrics

,contamination,accuracy,f1,f1_micro,f1_macro,precision,recall,roc_auc,model_score
0,100%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299
1,5%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299
2,10%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.058439


In [248]:
confusion_matrix_10 = {
            "contamination": "10%",
            "True Negative": 10291,
            "False positive": 5296,
            "False Negative": 891,
            "True Positive": 192
}

In [249]:
df_efc_confusion_matrix.loc[2] = confusion_matrix_10

In [250]:
df_efc_confusion_matrix

,contamination,True Negative,False positive,False Negative,True Positive
0,100%,10278,5309,857,226
1,5%,10273,5314,891,192
2,10%,10291,5296,891,192


### Train EFC With 100% Labeled Elliptic Data Set

In [251]:
# elliptic data set from reaml repo
X_train_100, X_test_100, y_train_100, y_test_100 = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/research_aml_elliptic/src/experiments/general_functions/elliptic_data_preprocessing.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)


In [252]:
# efc preps
intervals = get_intervals(X_train_100, 10)  # get discretization intervals from train set
X_train_100 = discretize(X_train_100, intervals)  # discretize train
X_test_100 = discretize(X_test_100, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '29894   NaN
29895   NaN
29896   NaN
29897   NaN
29898   NaN
         ..


In [253]:
# EFC's hyperparameters
Q = np.int64(X_test_100.values.max())
LAMBDA = 0.5  # pseudocount parameter

In [254]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train_100), Q, LAMBDA)  # train model

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:01<00:00, 86.45it/s]


In [255]:
y_predicted_100, energies = one_class_predict(np.array(X_test_100), coupling, h_i, cutoff, Q)  # test model

In [256]:
# colect results
confusion_matrix_100 = confusion_matrix(y_test_100, y_predicted_100)
print("Single-class results")
print('confusion_matrix', confusion_matrix_100)

Single-class results
confusion_matrix [[10249  5338]
 [  889   194]]


In [257]:
model_score_100 = calculate_model_score(y_true=y_test_100.values, y_pred=y_predicted_100, metric="f1")

In [258]:
y_true_100 = y_test_100.values
y_pred_100 = y_predicted_100

In [262]:
metric_dict_100 = {
        "contamination": "100%",
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
        "model_score": model_score_100,
    }

In [263]:
pprint(metric_dict_100)

{'accuracy': 0.6301139772045591,
 'contamination': '100%',
 'f1': np.float64(0.06829857963130856),
 'f1_macro': np.float64(0.4187761889998471),
 'f1_micro': np.float64(0.6301139772045591),
 'model_score': np.float64(0.058654572940287225),
 'precision': np.float64(0.04083107497741644),
 'recall': np.float64(0.20867959372114497),
 'roc_auc': np.float64(0.4340376219712416)}


In [264]:
df_efc_metrics.loc[3] = metric_dict_100

In [265]:
df_efc_metrics

,contamination,accuracy,f1,f1_micro,f1_macro,precision,recall,roc_auc,model_score
0,100%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299
1,5%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.068299
2,10%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.058439
3,100%,0.630114,0.068299,0.630114,0.418776,0.040831,0.20868,0.434038,0.058655


In [266]:
confusion_matrix_100 = {
            "contamination": "100%",
            "True Negative": 10249,
            "False positive": 5338,
            "False Negative": 889,
            "True Positive": 194
}

In [267]:
df_efc_confusion_matrix.loc[3] = confusion_matrix_100

In [268]:
df_efc_confusion_matrix

,contamination,True Negative,False positive,False Negative,True Positive
0,100%,10278,5309,857,226
1,5%,10273,5314,891,192
2,10%,10291,5296,891,192
3,100%,10249,5338,889,194


In [271]:
df_efc_metrics.to_csv('./efc_python/output/efc_metrics_v2.csv', index=False)

In [272]:
df_efc_confusion_matrix.to_csv('./efc_python/output/efc_confusion_matrix_v2.csv', index=False)

#################################################################################################################################

### Elliptic Data Set With Different Time Steps

In [273]:
# Import Elliptic data set and set variables
last_time_step = 42
last_train_time_step = 28
only_labeled = True

In [274]:
# '1': 1, -> class1 (illicit)
# '2': 0, -> class2 (licit)
# 'unknown': 2 -> dropped
X_train, X_test, y_train, y_test = run_elliptic_preprocessing_pipeline(last_train_time_step=last_train_time_step,
                                                                             last_time_step=last_time_step,
                                                                             only_labeled=only_labeled)

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/research_aml_elliptic/src/experiments/general_functions/elliptic_data_preprocessing.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes.replace({"class": {"1": 1, "2": 0, "unknown": 2}}, inplace=True)


In [275]:
intervals = get_intervals(X_train, 10)  # get discretization intervals from train set

In [276]:
X_train = discretize(X_train, intervals)  # discretize train
X_test = discretize(X_test, intervals)  # discretize test

/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data.iloc[:, feature].fillna(len(intervals[feature]), inplace=True)
/Users/kevinaraujo/repos/dissertation/PPCA-UnB-Dissertation/models/notebooks/efc_python/generic_discretize.py:35: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '25207   NaN
25208   NaN
25209   NaN
25210   NaN
25211   NaN
         ..


In [277]:
idx_abnormal = np.where(y_train == 1)[0]  # find abnormal samples indexes in the training set

In [278]:
X_train.drop(idx_abnormal, axis=0, inplace=True)  # remove abnormal samples from training (EFC trains with only benign instances)
y_train.drop(idx_abnormal, axis=0, inplace=True)  # remove the corresponding abonrmal training targets

In [279]:
# EFC's hyperparameters
Q = X_test.values.max()
LAMBDA = 0.5  # pseudocount parameter

In [280]:
coupling, h_i, cutoff, _, _ = one_class_fit(np.array(X_train), Q, LAMBDA)  # train model

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 166/166 [00:01<00:00, 88.54it/s]


In [282]:
y_predicted, energies = one_class_predict(np.array(X_test), coupling, h_i, cutoff, Q)  # test model

In [283]:
# colect results
confusion_matrix = confusion_matrix(y_test, y_predicted)
print("Single-class results")
print('confusion_matrix', confusion_matrix)

Single-class results
confusion_matrix [[8513 4323]
 [1622  212]]


In [284]:
model_score = calculate_model_score(y_true=y_test.values, y_pred=y_predicted, metric="f1")

In [285]:
y_true = y_test.values
y_pred = y_predicted

In [286]:
metric_dict = {
        "last_time_step": "42",
        "last_train_time_step": "28",
        "only_labeled": True,
        "accuracy": accuracy_score(y_true, y_pred),
        "f1": f1_score(y_true, y_pred, pos_label=1),
        "f1_micro": f1_score(y_true, y_pred, average="micro"),
        "f1_macro": f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "roc_auc": roc_auc_score(y_true, y_pred),
        "model_score": model_score,
    }

In [287]:
pprint(metric_dict)

{'accuracy': 0.5947511929107021,
 'f1': np.float64(0.06657246035484378),
 'f1_macro': np.float64(0.4038839403337059),
 'f1_micro': np.float64(0.5947511929107021),
 'last_time_step': '42',
 'last_train_time_step': '28',
 'model_score': np.float64(0.06657246035484378),
 'only_labeled': True,
 'precision': np.float64(0.046747519294377066),
 'recall': np.float64(0.11559432933478735),
 'roc_auc': np.float64(0.38940358411270376)}


In [288]:
columns_metrics = ["last_time_step", "last_train_time_step", "only_labeled", "accuracy", "f1", "f1_micro", "f1_macro", "precision", "recall", "roc_auc", "model_score"]

In [289]:
df_efc_metrics_v1 = pd.DataFrame(data=metric_dict, columns=columns_metrics, index=[0])

In [290]:
df_efc_metrics_v1

,last_time_step,last_train_time_step,only_labeled,accuracy,f1,f1_micro,f1_macro,precision,recall,roc_auc,model_score
0,42,28,True,0.594751,0.066572,0.594751,0.403884,0.046748,0.115594,0.389404,0.066572


In [291]:
confusion_matrix = {
            "last_time_step": "42",
            "last_train_time_step": "28",
            "only_labeled": True,
            "True Negative": 8513,
            "False positive": 4323,
            "False Negative": 1622,
            "True Positive": 212
}

In [295]:
columns = ["last_time_step", "last_train_time_step", "only_labeled", "True Negative", "False positive", "False Negative", "True Positive"]

In [297]:
df_efc_confusion_matrix = pd.DataFrame(data=confusion_matrix, columns=columns, index=[0])

In [298]:
df_efc_confusion_matrix

,last_time_step,last_train_time_step,only_labeled,True Negative,False positive,False Negative,True Positive
0,42,28,True,8513,4323,1622,212


In [ ]:
X_train, X_test, y_train, y_test

In [299]:
X_train

,time_step,trans_feat_0,trans_feat_1,trans_feat_2,trans_feat_3,trans_feat_4,trans_feat_5,trans_feat_6,trans_feat_7,trans_feat_8,...,agg_feat_62,agg_feat_63,agg_feat_64,agg_feat_65,agg_feat_66,agg_feat_67,agg_feat_68,agg_feat_69,agg_feat_70,agg_feat_71
0,0,9,7,2,3,0,3,0,0,8,...,0,0,1,1,2,3,0,4,0,0
1,0,9,7,3,3,0,3,0,0,7,...,0,0,1,1,2,2,0,4,0,0
2,0,7,0,1,0,0,0,0,8,7,...,0,0,1,1,1,0,0,1,0,0
3,0,7,0,1,0,0,0,0,8,7,...,2,3,0,0,1,0,0,1,0,0
4,0,3,0,1,1,0,0,0,1,2,...,0,1,1,1,1,1,1,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25200,9,1,3,4,0,0,0,0,5,2,...,5,7,0,0,1,0,0,1,0,0
25201,9,6,5,5,0,0,0,0,8,7,...,6,7,2,2,1,2,2,3,2,2
25202,9,8,5,5,0,0,0,0,9,8,...,0,0,1,1,1,0,0,1,0,0
25204,9,4,6,2,0,2,0,0,5,3,...,7,8,0,1,1,3,3,4,2,1
